In [1]:
print("Hi")

Hi


In [2]:
import torch
print(torch.cuda.is_available())

False


In [3]:
import torch
print(torch.__version__)

2.3.1+cpu


In [4]:
import numpy as np

In [5]:
import sys

# from bench_functions import micha, rose
# from botorch_gp import BotorchGp
from sklearn.metrics import mean_squared_error
model_directory = "C:/Projects/covnet2"
sys.path.append(model_directory)
from model import Model
from config import configs
import gc

In [6]:
def get_closest_samples(train_inputs, input, n):
    distances = np.linalg.norm(train_inputs - input, axis=1)

    # Get the indices of the 1000 smallest distances
    closest_indices = np.argsort(distances)[:n]

    # Select the 1000 closest samples
    return closest_indices

In [7]:
import pandas as pd
X = pd.read_csv("Data/X_train.csv")
y = pd.read_csv("Data/y_train.csv")

In [8]:
X_test = pd.read_csv("Data/X_test.csv")


In [9]:
X_test.head()

,id,trq_measured,oat,mgt,pa,ias,np,ng
0,0,76.2,29.50,648.0,303.2760,0.0000,99.83,96.77
1,1,63.3,18.75,595.4,464.8200,96.5625,100.01,93.61
2,2,87.3,2.50,644.4,503.5296,119.4375,99.92,96.87
3,3,85.4,0.25,630.7,458.4192,121.2500,100.04,96.04
4,4,73.1,21.25,625.9,626.3640,111.4375,100.17,95.67


In [10]:
X.head()

,id,trq_measured,oat,mgt,pa,ias,np,ng
0,0,54.100,2.00000,544.5000,212.1408,74.56250,89.18000,99.6400
1,1,49.625,24.22231,578.4844,1625.6400,30.35596,99.55273,91.3866
2,2,52.000,7.00000,566.1000,1912.9250,65.62500,100.14000,90.9600
3,3,62.400,7.25000,560.1000,277.0632,54.81250,90.64000,100.2800
4,4,62.900,23.25000,593.7000,53.6448,73.43750,99.91000,92.1700


In [11]:
y.head()

,id,faulty,trq_margin
0,0,1,-13.717745
1,1,0,1.791863
2,2,1,-13.944871
3,3,0,-0.017281
4,4,0,7.322404


In [12]:
ids = X["id"]
X_train = X.drop("id", axis=1)
X_test = X_test.drop("id", axis=1)

y_trq = y["trq_margin"]
y_faulty = y["faulty"]

In [13]:
len(y_trq)

742625

In [14]:
X_np = np.array(X_train)
X_test_np = np.array(X_test)
y_trq_np = np.array(y_trq)
y_faulty_np = np.array(y_faulty)

In [15]:
exp_train = X_np[:700000]
exp_test = X_np[700000:700010]
exp_y_train = y_trq_np[:700000]
exp_y_test = y_trq_np[700000:700010]
exp_yclass_train = y_faulty_np[:700000]
exp_yclass_test = y_faulty_np[700000:700010]

In [16]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
n_clusters=10
X_scaled = scaler.fit_transform(exp_train)
X_test_scaled = scaler.transform(exp_test)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_test_scaled)

# Get centroids and predict clusters
centroids = kmeans.cluster_centers_
cluster_labels = kmeans.predict(X_test_scaled)
pd.Series(cluster_labels).value_counts()

9    1
4    1
6    1
0    1
2    1
5    1
7    1
3    1
8    1
1    1
Name: count, dtype: int64

In [17]:
torch.tensor(5)

tensor(5)

In [18]:
cluster_labels_series = pd.Series(cluster_labels)
for i in range(n_clusters):
    indices = cluster_labels_series == i
    batch = X_test_scaled[indices]
    centroid = centroids[i]
    train_indices = get_closest_samples(X_scaled, centroid, 500)
    #for j in range(10):
    model = Model(X_scaled[train_indices], exp_y_train[train_indices], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    model.fit(epochs=4, verbose=True)
    torch.save(model, f'models/model_test.pth')
    predictions= []
    y_true=exp_y_test[indices]

    loaded_model = torch.load(f'models/model_test.pth')
    y_pred, lcb, ucb = loaded_model.predict(batch, pred_var=True)
    predictions.append(y_pred.cpu())
    print(f"Prediction: {y_pred}")
    print(ucb-y_pred)
    print(f"real_value_error: {y_pred-torch.tensor(y_true).view(-1,1)}")
    del loaded_model

    torch.cuda.empty_cache()
    gc.collect()


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using: cpu


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
Epoch: 100%|██████████| 4/4 [00:00<00:00, 29.20it/s, loss=-0.706, lr=1.000E-03, noise=8.27e-5]


Prediction: tensor([[0.3066]], dtype=torch.float64)
tensor([[0.2501]], dtype=torch.float64)
real_value_error: tensor([[0.0218]], dtype=torch.float64)


Epoch: 100%|██████████| 4/4 [00:00<00:00, 33.06it/s, loss=0.345, lr=1.000E-03, noise=1.69e-5]


...

Prediction: tensor([[1.5706]], dtype=torch.float64)
tensor([[0.8328]], dtype=torch.float64)
real_value_error: tensor([[0.0039]], dtype=torch.float64)


Epoch: 100%|██████████| 4/4 [00:00<00:00, 33.61it/s, loss=0.158, lr=1.000E-03, noise=2.95e-5]


Prediction: tensor([[17.3430]], dtype=torch.float64)
tensor([[0.5749]], dtype=torch.float64)
real_value_error: tensor([[-0.0073]], dtype=torch.float64)


Epoch: 100%|██████████| 4/4 [00:00<00:00, 33.90it/s, loss=-0.327, lr=1.000E-03, noise=1.43e-5]


Prediction: tensor([[6.9612]], dtype=torch.float64)
tensor([[0.0162]], dtype=torch.float64)
real_value_error: tensor([[-0.0006]], dtype=torch.float64)


In [16]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_np)
X_test_scaled = scaler.transform(X_test_np)


In [17]:
all_predictions=[]

for sample in X_test_scaled[20000:]:

    train_indices = get_closest_samples(X_scaled, sample, 500)
    predictions= []
    for j in range(10):
        try:
            model = Model(X_scaled[train_indices], y_trq_np[train_indices], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
            model.fit(epochs=4, verbose=False)
            y_pred= model.predict(sample)
            predictions.append(y_pred.cpu())
            del model
        except:
            predictions.append(False)
    all_predictions.append(predictions)

    torch.cuda.empty_cache()
    gc.collect()
import pickle
with open('predictions/y_pred_reg_20000_end.pkl', 'wb') as f:
    pickle.dump(all_predictions, f)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.sta

using: cpu


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\likelihoods\noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\botorch\models\utils\assorted.py:257: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


In [5]:
import pickle
with open('predictions/y_pred_reg_1000.pkl', 'rb') as f:
    array = pickle.load(f)

EOFError: Ran out of input

In [23]:
def closer_to_zero(a, b):
    if abs(a) < abs(b):
        return a,1
    else:
        return b,2

In [46]:
import torch.nn as nn
epochs = 1
model = Model(X_scaled, y_trq_np, noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
model.fit(epochs=epochs, verbose=True)

uncertainties=[]
predictions=[]
for i, test_sample in enumerate(X_test_scaled[:10]):
    if i%50==0:
        print(i)
    print(i)
    indicies = get_closest_samples(X_scaled, test_sample, 1000)
    # model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    # model.fit(epochs=epochs, verbose=False)
    # y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    # model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    # model.fit(epochs=epochs, verbose=False)
    # y_pred_2, lcb_2, ucb_2 = model.predict(test_sample, pred_var=True)
    # print(y_pred)
    # print(y_pred_2)
    for i in range(5):

        y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
        print(y_pred)
    print(ucb-y_pred)
    y_pred, v = closer_to_zero(y_pred, y_pred_2)
    if v ==1:
        uncertainty = ucb-y_pred
    else:
        uncertainty = ucb_2-y_pred
    se = uncertainty/1.96
    if se<0.01:
        se=0.01
    predictions.append(y_pred)
    uncertainties.append(se)
    
    

        

predictions = torch.stack(predictions)
uncertainties = torch.stack(uncertainties)
with open('predictions/reg_predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)
with open('predictions/reg_se.pkl', 'wb') as f:
    pickle.dump(uncertainties, f)

Epoch: 100%|██████████| 1/1 [01:03<00:00, 63.94s/it, loss=9.74, lr=1.000E-03, noise=1e-6]


0
0


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4411935125000 bytes.

In [27]:
predictions

tensor([[[ 8.5984]],

        [[ 1.5980]],

        [[-0.1050]],

        [[ 0.4020]],

        [[ 5.0188]],

        [[-3.3772]],

        [[16.3842]],

        [[14.6752]],

        [[ 9.3880]],

        [[10.2127]]], dtype=torch.float64)

In [39]:


from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(exp_train)
X_test_scaled = scaler.transform(exp_test)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_test_scaled)

# Get centroids and predict clusters
centroids = kmeans.cluster_centers_
cluster_labels = kmeans.predict(X_test_scaled)


In [20]:
import torch.nn as nn
errors= []
upper=[]
predictions=[]
for i, test_sample in enumerate(X_test_scaled):
    indicies = get_closest_samples(X_scaled, test_sample, 10000)
    model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-2)
    model.fit(epochs=2, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    predictions.append(y_pred.to('cpu').reshape(-1))
    upper.append(ucb.to('cpu'))
    print(f"Prediction: {y_pred}")
    print(f"Uncertainty: {ucb-y_pred}")
    print(f"real_value_error: {y_pred-torch.tensor(exp_y_test[i])}")
    errors.append(y_pred-torch.tensor(exp_y_test[i]))

mean_tensor = torch.stack(errors).mean(dim=0)

print(mean_tensor)

Prediction: tensor([[6.9089]], dtype=torch.float64)
Uncertainty: tensor([[2.0000e-05]], dtype=torch.float64)
real_value_error: tensor([[-0.0529]], dtype=torch.float64)
Prediction: tensor([[-2.4258]], dtype=torch.float64)
Uncertainty: tensor([[5.8726]], dtype=torch.float64)
real_value_error: tensor([[0.0134]], dtype=torch.float64)
Prediction: tensor([[9.9882]], dtype=torch.float64)
Uncertainty: tensor([[2.6858]], dtype=torch.float64)
real_value_error: tensor([[0.0030]], dtype=torch.float64)


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(


Prediction: tensor([[0.2890]], dtype=torch.float64)
Uncertainty: tensor([[1.1645]], dtype=torch.float64)
real_value_error: tensor([[0.0042]], dtype=torch.float64)
Prediction: tensor([[-73.8154]], dtype=torch.float64)
Uncertainty: tensor([[2.0000e-05]], dtype=torch.float64)
real_value_error: tensor([[0.0645]], dtype=torch.float64)
Prediction: tensor([[2.0928]], dtype=torch.float64)
Uncertainty: tensor([[1.8903]], dtype=torch.float64)
real_value_error: tensor([[0.0044]], dtype=torch.float64)
Prediction: tensor([[1.8151]], dtype=torch.float64)
Uncertainty: tensor([[6.5995]], dtype=torch.float64)
real_value_error: tensor([[0.2484]], dtype=torch.float64)
Prediction: tensor([[-19.3530]], dtype=torch.float64)
Uncertainty: tensor([[0.4016]], dtype=torch.float64)
real_value_error: tensor([[-0.0016]], dtype=torch.float64)
Prediction: tensor([[17.3280]], dtype=torch.float64)
Uncertainty: tensor([[2.0000e-05]], dtype=torch.float64)
real_value_error: tensor([[-0.0223]], dtype=torch.float64)
Predict

In [21]:
import torch.nn as nn
errors= []
upper=[]
predictions=[]
for i, test_sample in enumerate(X_test_scaled):
    indicies = get_closest_samples(X_scaled, test_sample, 5000)
    model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-2)
    model.fit(epochs=2, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    predictions.append(y_pred.to('cpu').reshape(-1))
    upper.append(ucb.to('cpu'))
    print(f"Prediction: {y_pred}")
    print(f"Uncertainty: {ucb-y_pred}")
    print(f"real_value_error: {y_pred-torch.tensor(exp_y_test[i])}")
    errors.append(y_pred-torch.tensor(exp_y_test[i]))
    

mean_tensor = torch.stack(errors).mean(dim=0)

print(mean_tensor)

Prediction: tensor([[6.9696]], dtype=torch.float64)
Uncertainty: tensor([[2.0000e-05]], dtype=torch.float64)
real_value_error: tensor([[0.0077]], dtype=torch.float64)
Prediction: tensor([[-2.3325]], dtype=torch.float64)
Uncertainty: tensor([[1.2905]], dtype=torch.float64)
real_value_error: tensor([[0.1067]], dtype=torch.float64)
Prediction: tensor([[9.9848]], dtype=torch.float64)
Uncertainty: tensor([[2.0070]], dtype=torch.float64)
real_value_error: tensor([[-0.0004]], dtype=torch.float64)
Prediction: tensor([[0.2856]], dtype=torch.float64)
Uncertainty: tensor([[1.5156]], dtype=torch.float64)
real_value_error: tensor([[0.0008]], dtype=torch.float64)
Prediction: tensor([[-73.8721]], dtype=torch.float64)
Uncertainty: tensor([[0.3973]], dtype=torch.float64)
real_value_error: tensor([[0.0078]], dtype=torch.float64)
Prediction: tensor([[2.0928]], dtype=torch.float64)
Uncertainty: tensor([[2.3880]], dtype=torch.float64)
real_value_error: tensor([[0.0044]], dtype=torch.float64)
Prediction: te

In [40]:
import torch.nn as nn
epochs = 8
errors= []

upper=[]
predictions=[]
for i, test_sample in enumerate(X_test_scaled):
    indicies = get_closest_samples(X_scaled, test_sample, 1000)
    model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    print(i)
        
    model.fit(epochs=8, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors.append(y_pred-torch.tensor(exp_y_test[i]))   

        # predictions.append(y_pred.to('cpu').reshape(-1))
        # upper.append(ucb.to('cpu'))
    print(f"Prediction: {y_pred}")
    print(f"Uncertainty: {ucb-y_pred}")
    print(f"real_value_error: {y_pred-torch.tensor(exp_y_test[i])}")
        

mean_tensor = torch.stack(errors).mean(dim=0)
print(mean_tensor)


0


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Prediction: tensor([[6.9618]], dtype=torch.float64)
Uncertainty: tensor([[0.0178]], dtype=torch.float64)
real_value_error: tensor([[-7.1135e-05]], dtype=torch.float64)
1
Prediction: tensor([[-2.3709]], dtype=torch.float64)
Uncertainty: tensor([[2.2972]], dtype=torch.float64)
real_value_error: tensor([[0.0683]], dtype=torch.float64)
2
Prediction: tensor([[9.9786]], dtype=torch.float64)
Uncertainty: tensor([[0.5514]], dtype=torch.float64)
real_value_error: tensor([[-0.0066]], dtype=torch.float64)
3
Prediction: tensor([[0.2732]], dtype=torch.float64)
Uncertainty: tensor([[0.4098]], dtype=torch.float64)
real_value_error: tensor([[-0.0116]], dtype=torch.float64)
4


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  warnings.warn(


Prediction: tensor([[-73.8809]], dtype=torch.float64)
Uncertainty: tensor([[2.0000e-05]], dtype=torch.float64)
real_value_error: tensor([[-0.0010]], dtype=torch.float64)
5
Prediction: tensor([[2.0951]], dtype=torch.float64)
Uncertainty: tensor([[0.8294]], dtype=torch.float64)
real_value_error: tensor([[0.0067]], dtype=torch.float64)
6
Prediction: tensor([[1.5952]], dtype=torch.float64)
Uncertainty: tensor([[1.1594]], dtype=torch.float64)
real_value_error: tensor([[0.0286]], dtype=torch.float64)
7
Prediction: tensor([[-19.3549]], dtype=torch.float64)
Uncertainty: tensor([[0.0660]], dtype=torch.float64)
real_value_error: tensor([[-0.0035]], dtype=torch.float64)
8
Prediction: tensor([[17.3434]], dtype=torch.float64)
Uncertainty: tensor([[0.6119]], dtype=torch.float64)
real_value_error: tensor([[-0.0069]], dtype=torch.float64)
9
Prediction: tensor([[7.4266]], dtype=torch.float64)
Uncertainty: tensor([[0.4590]], dtype=torch.float64)
real_value_error: tensor([[0.0110]], dtype=torch.float64)


In [38]:
import torch.nn as nn
epochs = 8
errors_1= []
errors_2= []
errors_3= []
errors_4= []
errors_5= []
errors_6= []
errors_7= []
errors_8= []
upper=[]
predictions=[]
for i, test_sample in enumerate(X_test_scaled):
    indicies = get_closest_samples(X_scaled, test_sample, 1000)
    model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    print(i)
        
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_1.append(y_pred-torch.tensor(exp_y_test[i]))
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_2.append(y_pred-torch.tensor(exp_y_test[i]))
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_3.append(y_pred-torch.tensor(exp_y_test[i]))
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_4.append(y_pred-torch.tensor(exp_y_test[i]))
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_5.append(y_pred-torch.tensor(exp_y_test[i]))
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_6.append(y_pred-torch.tensor(exp_y_test[i]))
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_7.append(y_pred-torch.tensor(exp_y_test[i]))
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    errors_8.append(y_pred-torch.tensor(exp_y_test[i]))        

        # predictions.append(y_pred.to('cpu').reshape(-1))
        # upper.append(ucb.to('cpu'))
    print(f"Prediction: {y_pred}")
    print(f"Uncertainty: {ucb-y_pred}")
    print(f"real_value_error: {y_pred-torch.tensor(exp_y_test[i])}")
        

mean_tensor = torch.stack(errors_1).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_2).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_3).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_4).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_5).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_6).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_7).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_8).mean(dim=0)

print(mean_tensor)

0


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [37]:
errors_8

[tensor([[0.0010]], dtype=torch.float64),
 tensor([[-0.0392]], dtype=torch.float64),
 tensor([[-0.0010]], dtype=torch.float64),
 tensor([[-0.0016]], dtype=torch.float64),
 tensor([[-0.3058]], dtype=torch.float64),
 tensor([[-0.0137]], dtype=torch.float64),
 tensor([[0.0025]], dtype=torch.float64),
 tensor([[-0.0441]], dtype=torch.float64),
 tensor([[0.0984]], dtype=torch.float64),
 tensor([[-0.0102]], dtype=torch.float64),
 tensor([[0.0026]], dtype=torch.float64),
 tensor([[0.2095]], dtype=torch.float64),
 tensor([[-0.0355]], dtype=torch.float64),
 tensor([[0.0005]], dtype=torch.float64),
 tensor([[0.0061]], dtype=torch.float64),
 tensor([[0.0048]], dtype=torch.float64),
 tensor([[0.0840]], dtype=torch.float64),
 tensor([[-0.0007]], dtype=torch.float64),
 tensor([[-3.7090e-05]], dtype=torch.float64),
 tensor([[-0.0234]], dtype=torch.float64),
 tensor([[-0.1021]], dtype=torch.float64),
 tensor([[-0.1298]], dtype=torch.float64),
 tensor([[0.0020]], dtype=torch.float64),
 tensor([[-0.0003

In [32]:
mean_tensor = torch.stack(errors_1).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_2).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_3).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_4).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_5).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_6).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_7).mean(dim=0)
print(mean_tensor)
mean_tensor = torch.stack(errors_8).mean(dim=0)

print(mean_tensor)

tensor([[-0.1082]], dtype=torch.float64)
tensor([[0.0470]], dtype=torch.float64)
tensor([[0.1152]], dtype=torch.float64)
tensor([[-0.5336]], dtype=torch.float64)
tensor([[-0.1843]], dtype=torch.float64)
tensor([[0.0013]], dtype=torch.float64)
tensor([[-0.0022]], dtype=torch.float64)
tensor([[-0.0010]], dtype=torch.float64)


In [ ]:
import torch.nn as nn
errors= []
upper=[]
predictions=[]
for i, test_sample in enumerate(X_test_scaled):
    indicies = get_closest_samples(X_scaled, test_sample, 1000)
    model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    predictions.append(y_pred.to('cpu').reshape(-1))
    upper.append(ucb.to('cpu'))
    print(f"Prediction: {y_pred}")
    print(f"Uncertainty: {ucb-y_pred}")
    print(f"real_value_error: {y_pred-torch.tensor(exp_y_test[i])}")
    errors.append(y_pred-torch.tensor(exp_y_test[i]))

mean_tensor = torch.stack(errors).mean(dim=0)

print(mean_tensor)

In [28]:
import torch.nn as nn
errors= []
upper=[]
predictions=[]
for i, test_sample in enumerate(X_test_scaled):
    indicies = get_closest_samples(X_scaled, test_sample, 1000)
    model = Model(X_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    model.fit(epochs=1, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    predictions.append(y_pred.to('cpu').reshape(-1))
    upper.append(ucb.to('cpu'))
    print(f"Prediction: {y_pred}")
    print(f"Uncertainty: {ucb-y_pred}")
    print(f"real_value_error: {y_pred-torch.tensor(exp_y_test[i])}")
    errors.append(y_pred-torch.tensor(exp_y_test[i]))

mean_tensor = torch.stack(errors).mean(dim=0)

print(mean_tensor)

Prediction: tensor([[6.9577]], dtype=torch.float64)
Uncertainty: tensor([[0.0110]], dtype=torch.float64)
real_value_error: tensor([[-0.0042]], dtype=torch.float64)
Prediction: tensor([[-2.4362]], dtype=torch.float64)
Uncertainty: tensor([[1.2614]], dtype=torch.float64)
real_value_error: tensor([[0.0030]], dtype=torch.float64)
Prediction: tensor([[9.9581]], dtype=torch.float64)
Uncertainty: tensor([[0.3205]], dtype=torch.float64)
real_value_error: tensor([[-0.0272]], dtype=torch.float64)
Prediction: tensor([[0.3032]], dtype=torch.float64)
Uncertainty: tensor([[0.1648]], dtype=torch.float64)
real_value_error: tensor([[0.0184]], dtype=torch.float64)
Prediction: tensor([[-73.8573]], dtype=torch.float64)
Uncertainty: tensor([[2.0000e-05]], dtype=torch.float64)
real_value_error: tensor([[0.0226]], dtype=torch.float64)
Prediction: tensor([[2.0835]], dtype=torch.float64)
Uncertainty: tensor([[0.5339]], dtype=torch.float64)
real_value_error: tensor([[-0.0049]], dtype=torch.float64)
Prediction: 

In [ ]:
mean_tensor = torch.stack(errors).mean(dim=0)

print(mean_tensor)

tensor([[0.0110]], dtype=torch.float64)


In [ ]:
cluster_labels_series = pd.Series(cluster_labels)
for i in range(50):
    indices = cluster_labels_series == i
    batch = X_test_scaled[indices]
    centroid = centroids[i]
    train_indices = get_closest_samples(X_scaled, centroid)
    model = Model(X_scaled[train_indices], exp_y_train[train_indices], noise=False, batch_size=1024, neurons=200, learning_rate=1e-4)
    model.fit(epochs=2, verbose=True)
    torch.save(model, f'models/model_test.pth')
    predictions= []
    for sample in X_test_scaled:
        loaded_model = torch.load(f'models/model_test.pth')
        y_pred, lcb, ucb = loaded_model.predict(sample.reshape(1,-1), pred_var=True)
        predictions.append(y_pred.cpu().reshape(1))
        print(f"Prediction: {y_pred}")
        print(f"Uncertainty: {ucb-y_pred}")
        print(f"real_value_error: {y_pred-torch.tensor(exp_y_test[i])}")
        del loaded_model

        torch.cuda.empty_cache()
        gc.collect()
    print(indices)

Epoch: 100%|██████████| 2/2 [00:01<00:00,  1.43it/s, loss=9.17, lr=1.000E-04, noise=3.42e-6]


Prediction: tensor([[5.4055]], dtype=torch.float64)
Uncertainty: tensor([[3.5921]], dtype=torch.float64)
real_value_error: tensor([[-1.5564]], dtype=torch.float64)
Prediction: tensor([[-0.6363]], dtype=torch.float64)
Uncertainty: tensor([[2.3670]], dtype=torch.float64)
real_value_error: tensor([[-7.5981]], dtype=torch.float64)
Prediction: tensor([[9.9097]], dtype=torch.float64)
Uncertainty: tensor([[1.5413]], dtype=torch.float64)
real_value_error: tensor([[2.9479]], dtype=torch.float64)
Prediction: tensor([[6.0141]], dtype=torch.float64)
Uncertainty: tensor([[4.7867]], dtype=torch.float64)
real_value_error: tensor([[-0.9478]], dtype=torch.float64)
Prediction: tensor([[-32.3952]], dtype=torch.float64)
Uncertainty: tensor([[10.9033]], dtype=torch.float64)
real_value_error: tensor([[-39.3571]], dtype=torch.float64)
Prediction: tensor([[0.4403]], dtype=torch.float64)
Uncertainty: tensor([[8.3269]], dtype=torch.float64)
real_value_error: tensor([[-6.5216]], dtype=torch.float64)
Prediction: 

KeyboardInterrupt: 

In [ ]:
centroids

array([[ 1.53858789, -1.315308  ,  1.10895045, ...,  1.39992477,
         0.6622477 ,  0.2403719 ],
       [ 0.22653748,  0.67254894, -0.22913171, ..., -0.46368323,
         0.68758428, -0.52633797],
       [ 0.63019948,  1.07857027,  0.87557534, ..., -1.34881169,
         0.64896926,  0.04613893],
       ...,
       [ 1.18476316,  1.93090512,  1.53291891, ..., -0.32348001,
         0.67357278,  0.27309396],
       [-0.62521868,  0.62915735, -1.12027216, ..., -0.83398767,
         0.67607484, -1.19286747],
       [-0.37158191,  0.74749806, -0.63527559, ..., -0.32669053,
         0.67698467, -0.93483203]])

In [ ]:
n_epochs = 10
model = Model(X_np[:700000], y_trq_np[:700000], noise=False, batch_size=1024, neurons=200, learning_rate=1e-4)
for epoch in range(n_epochs):
    model.fit(epochs=1, verbose=True)
    torch.save(model, f'models/model_{epoch}.pth')
    predictions= []
    for sample in X_np[700000:700100]:
        loaded_model = torch.load(f'models/model_{epoch}.pth')
        y_pred = loaded_model.predict(sample.reshape(1,-1))
        predictions.append(y_pred.cpu().reshape(1))

    mse = mean_squared_error(np.array(predictions), y_trq_np[700000:700100])
    print(f"{epoch}: {mse}")





Epoch: 100%|██████████| 1/1 [00:12<00:00, 12.66s/it, loss=10.9, lr=1.000E-04, noise=1e-6]
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


0: 9.949914947049579e-05


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
Epoch: 100%|██████████| 1/1 [00:12<00:00, 12.38s/it, loss=4.19, lr=1.000E-04, noise=1e-6]
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


1: 3.760940669708708e-05


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch: 100%|██████████| 1/1 [00:12<00:00, 12.33s/it, loss=3.79, lr=1.000E-04, noise=1e-6]
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


...

4: 0.0050524391476480714


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
Epoch: 100%|██████████| 1/1 [00:12<00:00, 12.34s/it, loss=3.82, lr=1.000E-04, noise=1e-6]
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


5: 0.09921345325621836


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch:   0%|          | 0/1 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
len(predictions)

122

In [ ]:
mse = mean_squared_error(predictions, y_trq_np[700000:700263])
print(f"{mse}")

0.0006916857582440312


In [ ]:
import torch
import numpy as np
from sklearn.metrics import mean_squared_error
import gc

predictions = []
for i, sample in enumerate(X_np[700000:700500]):
    # Load the model and move it to the appropriate device
    loaded_model = torch.load('models/model_1.pth', map_location=torch.device('cpu'))


    # Convert the sample to a PyTorch tensor and move it to the appropriate device
    sample_tensor = torch.tensor(sample).float().unsqueeze(0)


    y_pred, lcb, ucb = loaded_model.predict(sample_tensor, pred_var=True)  # Move prediction to CPU and convert to numpy

    predictions.append(y_pred.cpu().reshape(1))
    print(f"pred:{y_pred}")
    print(f"reak:{y_pred-y_trq_np[700000:700500][i]}")
    print(f"uncertainty:{ucb-y_pred}")

    # Clean up to free memory
    del loaded_model
    del sample_tensor
    torch.cuda.empty_cache()
    gc.collect()


# Convert predictions to numpy array
predictions = np.array(predictions).reshape(-1)

# Calculate MSE
mse = mean_squared_error(predictions, y_trq_np[700000:700500])
print(f"{epoch}: {mse}")

c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  warnings.warn(


pred:tensor([[6.9598]], dtype=torch.float64)
reak:tensor([[-0.0020]], dtype=torch.float64)
uncertainty:tensor([[2.0000e-05]], dtype=torch.float64)
pred:tensor([[-2.4383]], dtype=torch.float64)
reak:tensor([[0.0009]], dtype=torch.float64)
uncertainty:tensor([[5.6702]], dtype=torch.float64)
pred:tensor([[9.9809]], dtype=torch.float64)
reak:tensor([[-0.0043]], dtype=torch.float64)
uncertainty:tensor([[1.6373]], dtype=torch.float64)
pred:tensor([[0.2789]], dtype=torch.float64)
reak:tensor([[-0.0059]], dtype=torch.float64)
uncertainty:tensor([[1.7949]], dtype=torch.float64)
pred:tensor([[-73.8792]], dtype=torch.float64)
reak:tensor([[0.0007]], dtype=torch.float64)
uncertainty:tensor([[1.0528]], dtype=torch.float64)
pred:tensor([[2.0888]], dtype=torch.float64)
reak:tensor([[0.0004]], dtype=torch.float64)
uncertainty:tensor([[2.3447]], dtype=torch.float64)
pred:tensor([[1.5661]], dtype=torch.float64)
reak:tensor([[-0.0006]], dtype=torch.float64)
uncertainty:tensor([[5.4194]], dtype=torch.floa

KeyboardInterrupt: 

In [ ]:
predictions=[]
for sample in X_np[700000:700500]:
    loaded_model = torch.load(f'models/model_1.pth')
    y_pred = loaded_model.predict(sample.reshape(1,-1))
    predictions.append(y_pred.cpu().reshape(1))
mse = mean_squared_error(np.array(predictions), y_trq_np[700000:700500])
print(f"{epoch}: {mse}")

c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 

In [ ]:
mse = mean_squared_error(np.array(predictions), y_trq_np[700000:700500])
print(f"{epoch}: {mse}")

In [ ]:
torch.save(model, 'models/test_model.pth')

In [ ]:
loaded_model = torch.load('models/test_model.pth')

In [ ]:
y_pred= loaded_model.predict(X_np[700002:700003])

In [ ]:
y_pred

tensor([[9.5952]], device='cuda:0', dtype=torch.float64)

In [ ]:
y_trq_np[700002]

9.985207392177664

In [ ]:
d=np.array([5,67])
np.sum(d)

np.int64(72)

In [ ]:
len(X_np)


742625

In [ ]:
torch.cuda.is_available()

True

In [ ]:
batch_sizes = [64, 128, 256, 512, 1048]
repeats = 5
pcas = ["enabled", "disabled"]

In [ ]:
X_np
X_np.shape

(742625, 7)

In [ ]:
mse=2

In [ ]:
def get_closest_samples(train_inputs, input):
    distances = np.linalg.norm(train_inputs - input, axis=1)

    # Get the indices of the 1000 smallest distances
    closest_indices = np.argsort(distances)[:10000]

    # Select the 1000 closest samples
    return closest_indices

In [ ]:
from scipy.spatial.distance import mahalanobis
def get_closest_samples_mahalanobis(train_inputs, input):
    # Compute the covariance matrix of train_inputs
    cov_matrix = np.cov(train_inputs, rowvar=False)

    # Compute the inverse covariance matrix (if not singular)
    try:
        inv_cov_matrix = np.linalg.inv(cov_matrix)
    except np.linalg.LinAlgError:
        # Handle the case where the covariance matrix is singular
        inv_cov_matrix = np.eye(train_inputs.shape[1])  # Use identity matrix as a fallback

    # Compute Mahalanobis distances
    mahalanobis_distances = []
    for sample in train_inputs:
        dist = mahalanobis(sample, input, inv_cov_matrix)
        mahalanobis_distances.append(dist)

    # Get the indices of the 1000 smallest distances
    closest_indices = np.argsort(mahalanobis_distances)[:4050]

    # Return the indices of the closest samples
    return closest_indices

In [22]:

predictions = []
upper = []
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_np[:700000])
X_np_scaled = scaler.transform(X_np)
for i, test_sample in enumerate(X_np_scaled[700000:700010]):
    indicies = get_closest_samples(X_np_scaled[:700000], test_sample)
    model = Model(X_np_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-2)
    model.fit(epochs=2, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    predictions.append(y_pred.to('cpu'))
    upper.append(ucb.to('cpu'))
    mse = mean_squared_error(np.array(y_pred.to('cpu')), np.array(y_trq_np[700000:700010][i]).reshape(1,1))
    print(mse)
    print(y_pred.to('cpu')-ucb.to('cpu'))

predictions = np.array(predictions).reshape(-1)
mse = mean_squared_error(predictions, y_trq_np[700000:700010])
print(mse)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.sta

using: cpu


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\

RuntimeError: Numpy is not available

In [ ]:
predictions = []
upper = []
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_np[:700000])
X_np_scaled = scaler.transform(X_np)
for i, test_sample in enumerate(X_np_scaled[700000:700100]):
    indicies = get_closest_samples_mahalanobis(X_np_scaled[:700000], test_sample)
    model = Model(X_np_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    model.fit(epochs=30, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    predictions.append(y_pred.to('cpu'))
    upper.append(ucb.to('cpu'))
    mse = mean_squared_error(np.array(y_pred.to('cpu')), np.array(y_trq_np[700000:700100][i]).reshape(1,1))
    print(mse)
    print(y_pred.to('cpu')-ucb.to('cpu'))

predictions = np.array(predictions).reshape(-1)
mse = mean_squared_error(predictions, y_trq_np[700000:700100])
print(mse)

In [25]:
upper-predictions

array([1.82109832e-02, 1.53442318e+00, 2.69240206e-01, 7.41821727e-03,
       8.39102007e-03, 4.11864508e-01, 1.17057637e+00, 7.42744247e-02,
       9.09095602e-01, 4.03196193e-01, 9.29225487e-01, 2.73146409e-01,
       2.22771321e-01, 2.00000000e-05, 4.28641171e-01, 6.16244775e-01,
       5.74825217e-01, 1.95233212e-02, 7.02649719e-01, 3.66861982e-01,
       1.47994757e+00, 2.00000000e-05, 4.33813366e-01, 7.57228132e-01,
       1.35393295e-02, 2.59754865e-01, 1.43452497e-01, 1.47754756e-01,
       4.19194391e-01, 2.98340375e-01])

In [38]:
def create_bootstrap_sample(data, n_samples):
    n_rows = data.shape[0]
    bootstrap_samples = []
    for _ in range(n_samples):
        indices = np.random.choice(n_rows, size=5000, replace=True)
        bootstrap_sample = data[indices]
        bootstrap_samples.append(bootstrap_sample)
    return bootstrap_samples

In [40]:
samples = create_bootstrap_sample(np.concatenate((X_np[:700000], y_trq_np[:700000].reshape(-1,1)), axis=1),10)
predictions = []
for sample in samples:
    X = sample[:,:7]
    y = sample[:,7]
    model = Model(X, y, noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
    model.fit(epochs=10, verbose=True)
    y_pred, lcb, ucb = model.predict(X_np[700000:700500], pred_var=True)
    print(mean_squared_error(np.array(y_pred.to('cpu')).reshape(-1,1), y_trq_np[700000:700500]))


    predictions.append(y_pred)





Epoch: 100%|██████████| 10/10 [00:01<00:00,  7.00it/s, loss=18.1, lr=1.000E-03, noise=1e-6]
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


0.05364855803823902


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch: 100%|██████████| 10/10 [00:01<00:00,  8.06it/s, loss=5.82e+4, lr=1.000E-03, noise=1e-6]
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\

123.74193137562021


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch:  90%|█████████ | 9/10 [00:01<00:00,  9.05it/s, loss=17.9, lr=1.000E-03, noise=1e-6]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
Epoch: 100%|██████████| 10/10 [00:01<00:00,  8.85it/s, loss=17.6, lr=1.000E-03, noise=1e-6]
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\L

...

0.06034403132676745


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch:  30%|███       | 3/10 [00:00<00:00,  8.55it/s, loss=22.9, lr=1.000E-03, noise=1e-6]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
Epoch: 100%|██████████| 10/10 [00:01<00:00,  8.01it/s, loss=13, lr=1.000E-03, noise=1e-6] 
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Li

0.18973436037912206


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch:  30%|███       | 3/10 [00:00<00:00,  8.77it/s, loss=18.9, lr=1.000E-03, noise=1e-6]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
Epoch: 100%|██████████| 10/10 [00:01<00:00,  8.73it/s, loss=12.6, lr=1.000E-03, noise=1e-6]


0.0668374973153036


In [ ]:
all_predictions = np.array(predictions)
aggregated_predictions = np.mean(all_predictions, axis=0)
print(mean_squared_error(aggregated_predictions, y_trq_np[700000:]))

In [35]:
len(samples)

100

In [17]:
lowest_mse=1 
X_valid_np = np.array(X_train[700000:])
X_test_np = np.array(X_train[100000:100001])
model = Model(X_np, y_trq_np, noise=False, batch_size=1024, neurons=200, learning_rate=1e-3)
for epoch in range(100):
    model.fit(epochs=1, verbose=True)
    
    # y_pred_reg_test = model.predict(X_test_np)
    # y_pred_reg_test = y_pred_reg_test.to("cpu")
    # mse_test = mean_squared_error(y_pred_reg_test, np.array(y_trq[100000:105000]))
    # print(mse_test)
    # y_pred_reg = model.predict(X_valid_np)
    # y_pred_reg = y_pred_reg.to("cpu")
    # mse = mean_squared_error(y_pred_reg, np.array(y_trq[700000:705000]))
    # print(mse)
    # if epoch %10 ==0 or mse<lowest_mse:
    #     model_name= f"models/model_{epoch}.pth"
    #     torch.save(model, model_name)
    #     if mse<lowest_mse:
    #         lowest_mse=mse


using: cuda:0


c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(
c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\linear_operator\utils\cholesky

In [28]:
X_test_np = np.array(X_train[100000:101000])

In [23]:
y_trq[100000]

9.661960354567126

In [24]:
y_pred_reg_valid

tensor([[10.0186]], device='cuda:0', dtype=torch.float64)

In [29]:
predictions=[]
for i in X_test_np:
    y_pred_reg_valid = model.predict(i)
    predictions.append(y_pred_reg_valid.to('cpu'))
predictions = np.array(predictions).reshape(-1)

In [30]:
mse = mean_squared_error(predictions, np.array(y_trq[100000:101000]))

In [31]:
mse

3025.9186496036523

In [26]:
predictions

array([[[10.01860018]]])

In [18]:
X_test_np.shape

(1, 7)

In [19]:

X_np.shape

(10000, 7)

In [20]:
X_train.shape

(742625, 7)

In [21]:
X_valid_np.shape

(42625, 7)

In [32]:
y_pred_reg_valid

tensor([[ 6.2215],
        [-3.0985],
        [ 9.4272],
        ...,
        [-8.3928],
        [ 7.6205],
        [ 5.0785]], device='cuda:0', dtype=torch.float64)

In [34]:
X_valid_np = np.array(X_train[700000:])
y_pred_reg_valid = model.predict(X_test_np)

In [23]:
len(y_pred_reg_valid)

42625

In [20]:
predictions =np.array(predictions).reshape(-1)

In [36]:
y_pred_reg_valid.reshape(-1).shape

torch.Size([42625])

In [39]:
np.array(y_trq_np[700000:]).shape

(0,)

In [40]:
y_trq_np.shape

(5000,)

In [41]:
mse = mean_squared_error(np.array(y_pred_reg_valid.reshape(-1).to('cpu')), np.array(y_trq[700000:]))

In [42]:
mse

1.891956202372026

In [21]:
y_pred_reg_test

tensor([[  9.7287],
        [  4.4659],
        [ 10.9164],
        [  1.3744],
        [ 12.1732],
        [ 13.6505],
        [  8.6195],
        [  2.6678],
        [  0.4577],
        [-15.0898],
        [-11.3290],
        [ -0.3779],
        [ 19.3056],
        [  8.5827],
        [ 11.9428],
        [ -2.1250],
        [ -0.0356],
        [ -3.6832],
        [-12.5571],
        [  6.4646],
        [  6.2279],
        [  5.6280],
        [ -0.9589],
        [ -5.8614],
        [  6.2193],
        [  7.6595],
        [  6.0361],
        [  7.6964],
        [ -8.1492],
        [-18.1537],
        [ 12.7509],
        [ -6.5269],
        [  5.6578],
        [  1.9772],
        [ -1.7935],
        [ -7.6769],
        [ -3.1195],
        [  2.5145],
        [  9.0783],
        [  3.2320],
        [ 11.9081],
        [ -6.0328],
        [  7.2724],
        [ 17.5959],
        [  5.2186],
        [  1.8403],
        [ -7.5535],
        [  2.8366],
        [ 11.2584],
        [ -4.7096]],

In [ ]:
model = torch.load('model.pth')

In [64]:
torch.save(model, 'model.pth')

In [67]:
X_test_np = np.array(X_train[10000:20000])
y_pred = model.predict(X_test_np)
y_pred = y_pred.to("cpu")
mse = mean_squared_error(y_pred, np.array(y_trq[10000:20000]))
mse

0.02168344920155582

In [42]:
without_pca

[0.23349293415623096,
 0.1909339338111983,
 0.007933209249155575,
 0.029223683480669723,
 0.03361694145177521,
 0.10668213075885415,
 0.030489042080994712,
 0.015514424225184833,
 0.016535532116858202,
 1.2902604334861452]

In [24]:
model = Model(
    X_np, y_trq_np, noise=False, batch_size=512, neurons=50, learning_rate=1e-2
)
model.fit(epochs=100, verbose=True)

c:\Users\janni\anaconda3\envs\cuda\Lib\site-packages\gpytorch\likelihoods\noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(
c:\Users\janni\anaconda3\envs\cuda\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
Epoch:   0%|          | 0/100 [00:00<?, ?it/s]c:\Users\janni\anaconda3\envs\cuda\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
Epoch:   1%|          | 1/100 [00:00<00:46,  2.14it/s, loss=66.6, lr=1.000E-02, noise=1e-6]c:\Users\janni\anaconda3\envs\cuda

In [26]:

output = model(np.array(X_train[10000:15000]))

TypeError: 'Model' object is not callable

In [25]:
y_pred = model.predict(np.array(X_train[10000:15000]))

In [24]:
from sklearn.metrics import mean_squared_error
y_pred = y_pred.to("cpu")
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))

In [25]:
mse

0.030111247707689363

In [23]:
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(X_np, y_trq_np)
y_pred = lin.predict(np.array(X_train[10000:15000]))
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))
mse

5.320451391219264

In [27]:
class_model = Model(
    X_np, y_faulty_np, noise=False, batch_size=512, neurons=50, learning_rate=1e-2
)
class_model.fit(epochs=150, verbose=True)

c:\Users\je009447\AppData\Local\miniconda3\envs\bayesnewton\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch:   0%|          | 0/150 [00:00<?, ?it/s]

c:\Users\je009447\AppData\Local\miniconda3\envs\bayesnewton\lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\je009447\AppData\Local\miniconda3\envs\bayesnewton\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


In [28]:
y_pred = model.predict(np.array(X_train[10000:15000]))
y_pred = np.round(y_pred)

In [ ]:
y_pred, lcb, ucb = model.predict(np.array(X_train[10000:15000]), pred_var=True)

In [30]:
y_pred

tensor([[-7.],
        [-6.],
        [-4.],
        ...,
        [ 8.],
        [14.],
        [-4.]], dtype=torch.float64)

In [38]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_pred, np.array(y_faulty[10000:15000]))
acc

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [32]:
from xgboost import XGBRegressor
xgboost = XGBRegressor()
xgboost.fit(X_np, y_trq_np)
y_pred_xg = xgboost.predict(np.array(X_train[10000:15000]))
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))
print(mse)

0.1910663424724587


In [35]:
y_trq_np.reshape(-1,1).shape

(10000, 1)

In [36]:
X_np_class = np.concatenate((X_np, y_trq_np.reshape(-1,1)), axis=1)

In [57]:
from xgboost import XGBClassifier

xgboost = XGBClassifier()
xgboost.fit(X_np, y_faulty_np)
y_pred_xg = xgboost.predict(np.array(X_train[700000:705000]))
acc = accuracy_score(y_pred_xg, np.array(y_faulty[700000:705000]))
print(acc)

ValueError: Feature shape mismatch, expected: 7, got 8

In [43]:
X_train = np.concatenate((X_train[700000:705000], np.array(y_trq[700000:705000]).reshape(-1,1)), axis=1)

In [44]:
X_train.shape

(5000, 8)

In [46]:
X_np_class.shape

(10000, 8)

In [47]:
y_faulty_np.shape

(10000,)

In [49]:
np.array(y_faulty[700000:705000])

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [53]:
y_pred_xg

array([], dtype=int32)

In [54]:
np.array(X_train[700000:705000])

array([], shape=(0, 8), dtype=float64)

In [ ]:
X_np = np.array(X_train)
X_test_np = np.array(X_test)
y_trq_np = np.array(y_trq)
y_faulty_np = np.array(y_faulty)

In [14]:
len(X_np)

742625

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgboost = XGBClassifier()
xgboost.fit(X_np, y_faulty_np.reshape(-1,1))
y_pred_class = xgboost.predict(np.array(X_test))


In [18]:
y_pred_test = xgboost.predict(np.array(X_np))
acc = accuracy_score(y_pred_test, y_faulty_np)
print(acc)

0.998370644672614


In [20]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split



# Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_clf, 
                           param_grid=param_grid, 
                           scoring='accuracy', 
                           cv=3, 
                           verbose=1, 
                           n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_np[:700000], y_faulty_np[:700000])

# Print the best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

# Evaluate the model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_np[700000:])
accuracy = accuracy_score(y_faulty_np[700000:], y_pred)
print(f"Test Set Accuracy: {accuracy}")

Fitting 3 folds for each of 32 candidates, totalling 96 fits


c:\Users\janni\anaconda3\envs\CUDA\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:48:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}
Best Score: 0.9922314281999688
Test Set Accuracy: 0.9916715542521994


In [21]:

y_pred_class = best_model.predict(np.array(X_test))


In [22]:
y_pred_class

array([1, 0, 1, ..., 1, 0, 0])

In [24]:
import pickle
predictions_path = os.path.join('predictions' 'y_pred_class.pkl')
with open('predictions/y_pred_class.pkl', 'wb') as f:
    pickle.dump(y_pred_class, f)

In [16]:
import numpy as np

In [20]:
predictions = []
upper = []
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_np)
X_np_scaled = scaler.transform(X_np)
X_test_np_scaled = scaler.transform(X_test_np)
for i, test_sample in enumerate(X_test_np_scaled[:5000]):
    indicies = get_closest_samples_mahalanobis(X_np_scaled, test_sample)
    model = Model(X_np_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-2)
    model.fit(epochs=2, verbose=False)
    y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
    predictions.append(y_pred.to('cpu').reshape(-1))
    upper.append(ucb.to('cpu'))
    if i%500 ==0:
        print(i)




c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


0


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(


500
1000
1500
2000
2500
3000
3500
4000
4500


In [21]:
import pickle
with open('predictions/y_pred_reg_5000.pkl', 'wb') as f:
    pickle.dump(predictions, f)

with open('predictions/y_pred_conf_5000.pkl', 'wb') as f:
    pickle.dump(upper, f)

In [62]:
predictions3 = []
upper3 = []
faulty = []
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_np)
X_np_scaled = scaler.transform(X_np)
X_test_np_scaled = scaler.transform(X_test_np)
for i, test_sample in enumerate(X_test_np_scaled[6192:8000]):
    try:
        indicies = get_closest_samples_mahalanobis(X_np_scaled, test_sample)
        model = Model(X_np_scaled[indicies], y_trq_np[indicies], noise=False, batch_size=1024, neurons=200, learning_rate=1e-2)
        model.fit(epochs=2, verbose=False)
        y_pred, lcb, ucb = model.predict(test_sample, pred_var=True)
        predictions3.append(y_pred.to('cpu').reshape(-1))
        upper3.append(ucb.to('cpu'))
        if i%500 ==0:
            print(i)
    except:
        faulty.append(test_sample)
        continue

c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


0


c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-10.
  warnings.warn(
c:\Users\janni\anaconda3\envs\datascience\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(


500
1000
1500


In [15]:
faulty


NameError: name 'faulty' is not defined

In [28]:
len(predictions3)

881

In [29]:
len(upper3)

881

In [64]:
import pickle
with open('predictions/y_pred_reg_8000.pkl', 'wb') as f:
    pickle.dump(predictions3, f)

with open('predictions/y_pred_conf_8000.pkl', 'wb') as f:
    pickle.dump(upper3, f)

In [28]:
predictions

array([ 9.13918631,  1.89735088, -3.53644523, -3.19939769,  5.51870315,
        1.43063301, 16.02386305, 14.87186913,  9.8996958 , 10.25744746])

In [ ]:
lin = LinearRegression()
lin.fit(X_np, y_trq_np)
y_pred = lin.predict(np.array(X_train[10000:15000]))
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))
mse

In [ ]:
error_margin = upper_bound - y_pred_reg
se = error_margin/1.96
# se is the standard deviation

In [ ]:
output_data = {}
for i in ids:
    {
        "class": y_pred_class[i - 1],
        "class_conf": y_pred_class_conf[i - 1],
        "pdf_type": "norm",
        "pdf_args": {"loc": y_pred_reg, "scale": y_pred_reg_dev},
    }
    output_data[i] = {}

In [ ]:
import json
with open("output.json", "w") as json_file:
    json.dump(output_data, json_file, indent=4)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split



# Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_clf, 
                           param_grid=param_grid, 
                           scoring='accuracy', 
                           cv=3, 
                           verbose=1, 
                           n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(exp_train,exp_yclass_train)

# Print the best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

# Evaluate the model on the test set
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
y_pred = best_model.predict(exp_test)
accuracy = accuracy_score(exp_yclass_test, y_pred)
print(f"Test Set Accuracy: {accuracy}")
xgb_clf_best = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree']
)

xgb_clf_best.fit(X_np, y_faulty_np)
y_pred_class = xgb_clf_best.predict(X_test_np)
import pickle
with open('predictions/class_predictions.pkl', 'wb') as f:
    pickle.dump(y_pred_class, f)